In [ ]:
!spacy download es_core_news_sm

     |████████████████████████████████| 16.2MB 3.9MB/s 
  Created wheel for es-core-news-sm: filename=es_core_news_sm-2.2.5-cp37-none-any.whl size=16172936 sha256=f22ec4d952b4c37693dde49d9ebe137abf28e50f79b3c52810a0e244b5b8f600
  Stored in directory: /tmp/pip-ephem-wheel-cache-klxn95lw/wheels/05/4f/66/9d0c806f86de08e8645d67996798c49e1512f9c3a250d74242
Successfully built es-core-news-sm
✔ Download and installation successful
You can now load the model via spacy.load('es_core_news_sm')


In [ ]:
import spacy
nlp = spacy.load('es_core_news_sm')

In [ ]:
# Procesamiento texto

#pip install TextBlob
#pip install spacy
import nltk
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('punkt')
from nltk.stem.porter import PorterStemmer
nltk.download('')
import pandas as pd
import re
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer, TfidfVectorizer
from nltk.stem import PorterStemmer
from nltk import SnowballStemmer
from textblob import TextBlob
import string



# Modelos
from sklearn.model_selection import train_test_split
from sklearn import tree
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn import svm
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier

# Métricas
from sklearn.model_selection import cross_val_score
from sklearn import metrics
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV

from sklearn.metrics import f1_score
from scipy.stats import pearsonr
from sklearn.model_selection import StratifiedKFold

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Error loading : Package '' not found in index


In [ ]:
df = pd.read_csv("train 2.csv")
test = pd.read_csv("test.csv")

In [ ]:
replacements = (
        ("á", "a"),
        ("é", "e"),
        ("í", "i"),
        ("ó", "o"),
        ("ú", "u"),
    )

# Procesamiento train

In [ ]:
errores = []
lemmas = []
for i in range(len(df)):
        #tokenizar
        tokens = nltk.word_tokenize(df.loc[:,'comment'].iloc[i])
        #elimino signos de puntuación y ruido
        #table = str.maketrans('', '', string.punctuation)
        tokens = ' '.join(tokens)
        tokens = tokens.lower()
        punct = string.punctuation
        for c in punct:
            tokens = tokens.replace(c, " ")
        for a1, b1 in replacements:
            tokens = tokens.replace(a1, b1).replace(a1.upper(), b1.upper())
            stripped = tokens.split()
        #stripped = [w.translate(table) for w in tokens] #intento sacar los signos de puntuación pero no puedo
        #stripped = re.sub('[%s]' % re.escape(string.punctuation), ' ', tokens)

        words = [word for word in stripped if word.isalpha()]


        #eliminar stopwords
        stop_words = set(stopwords.words('spanish'))
        #words = [w for w in stripped if not w in stop_words]
        words = [word for word in stripped if not word in stop_words]

        #stemming
        porter = PorterStemmer()
        for i in range(len(words)):
            words[i] = porter.stem(words[i])

        #stemmed = [porter.stem(word) for word in words]

        #lemmatizar 
        words = ' '.join(words)
        doc = nlp(words)
        lemmas1 = [tok.lemma_.lower() for tok in doc]
        lemmas.append(' '.join(lemmas1))
lemmas

#extraction
try:
    vectorizer = CountVectorizer()
    vectorizer.fit(lemmas)
    X_bag_of_words = vectorizer.transform(lemmas)
except ValueError:
    errores.append(lemmas1)

In [ ]:
tfidf_vectorizer = TfidfVectorizer()
tfidf_vectorizer.fit(lemmas)

tfidf = tfidf_vectorizer.transform(lemmas)

y_train = df['toxicity']
X_train = tfidf

# TEST

In [ ]:
errores = []
lemmas = []
for i in range(len(test)):
        #tokenizar
        tokens = nltk.word_tokenize(test.loc[:,'comment'].iloc[i])
        #elimino signos de puntuación y ruido
        #table = str.maketrans('', '', string.punctuation)
        tokens = ' '.join(tokens)
        tokens = tokens.lower()
        punct = string.punctuation
        for c in punct:
            tokens = tokens.replace(c, " ")
        for a1, b1 in replacements:
            tokens = tokens.replace(a1, b1).replace(a1.upper(), b1.upper())
            stripped = tokens.split()
        #stripped = [w.translate(table) for w in tokens] #intento sacar los signos de puntuación pero no puedo
        #stripped = re.sub('[%s]' % re.escape(string.punctuation), ' ', tokens)

        words = [word for word in stripped if word.isalpha()]


        #eliminar stopwords
        stop_words = set(stopwords.words('spanish'))
        #words = [w for w in stripped if not w in stop_words]
        words = [word for word in stripped if not word in stop_words]

        #stemming
        porter = PorterStemmer()
        for i in range(len(words)):
            words[i] = porter.stem(words[i])

        #stemmed = [porter.stem(word) for word in words]

        #lemmatizar 
        words = ' '.join(words)
        doc = nlp(words)
        lemmas1 = [tok.lemma_.lower() for tok in doc]
        lemmas.append(' '.join(lemmas1))
lemmas

#extraction
try:
    vectorizer = CountVectorizer()
    vectorizer.fit(lemmas)
    X_bag_of_words = vectorizer.transform(lemmas)
except ValueError:
    errores.append(lemmas1)

In [ ]:
#TEST

tfidf_test = tfidf_vectorizer.transform(lemmas)

x_test = tfidf_test

# DETECCIÓN DE TOXICIDAD

# Logistic Regression

In [ ]:
clf_log = LogisticRegression(random_state = 0, C=10,penalty='l2'
                                 ,solver = 'liblinear',class_weight="balanced")
clf_log.fit(X_train, y_train)
y_pred_log = clf_log.predict(x_test)

In [ ]:
print(y_pred_log)

[0 1 0 0 0 0 1 0 1 0 0 0 0 0 0 0 1 0 0 0 1 1 0 0 0 1 1 1 0 0 0 0 1 0 0 0 1
 0 1 1 1 1 1 0 0 1 0 0 1 0 0 1 0 0 1 0 0 0 1 0 0 1 1 0 1 0 1 0 0 0 0 1 1 0
 0 0 0 1 1 0 1 0 0 1 1 1 1 0 0 0 0 1 0 0 0 0 0 0 0 1 0 0 1 0 1 0 1 0 1 0 1
 0 0 1 1 0 0 0 0 0 0 0 0 1 1 1 0 1 0 0 0 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0
 0 0 1 0 0 0 1 0 0 1 0 0 0 0 0 0 0 0 1 0 0 0 1 0 0 0 1 1 0 0 1 0 0 0 0 1 0
 1 0 0 0 0 1 0 1 0 0 0 1 0 0 0 0 1 0 0 0 0 1 0 1 1 0 1 0 0 0 0 1 1 1 0 1 1
 0 1 0 0 1 1 0 0 0 0 0 1 0 1 1 1 0 0 0 0 0 0 0 0 1 1 0 1 1 1 0 0 0 0 1 0 1
 1 0 0 1 0 1 0 0 0 0 0 0 0 1 0 0 1 1 1 0 1 0 0 0 0 0 1 0 0 0 0 1 1 0 1 1 0
 1 1 1 1 0 0 0 1 0 0 0 0 1 0 0 0 1 0 0 0 0 0 0 1 1 0 0 0 1 0 1 0 0 1 0 1 0
 0 0 1 1 0 0 0 0 0 0 0 0 1 0 0 1 0 1 0 0 0 1 0 0 0 1 0 0 0 0 1 0 1 0 0 0 1
 0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 1 1 0 1 1 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 1
 0 1 1 0 0 1 0 0 0 1 0 0 1 0 0 0 1 1 1 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 1 0
 0 0 0 0 0 0 0 0 1 0 1 0 0 1 0 0 0 1 0 0 0 0 1 1 0 1 0 0 1 0 0 0 0 0 0 0 0
 1 1 0 0 0 1 0 0 0 0 0 1 

# Bagging

In [ ]:
from sklearn.ensemble import BaggingClassifier

m_bagging = BaggingClassifier(base_estimator=LogisticRegression(random_state = 0, C=10,penalty='l2'
                                 ,solver = 'liblinear',class_weight="balanced"),
                               n_estimators=10)

m_bagging.fit(X_train, y_train)
y_pred_bag = m_bagging.predict(x_test)

In [ ]:
print(m_pred_bag)

[0 1 0 0 1 0 1 0 1 0 0 0 0 0 0 0 1 0 0 0 0 1 0 0 0 1 1 0 0 0 0 0 1 0 0 0 1
 0 1 1 0 1 0 0 0 1 0 0 1 0 0 1 0 0 1 0 0 0 1 0 0 1 1 0 1 0 1 0 0 0 0 1 1 0
 0 0 0 1 1 0 1 0 0 0 0 1 1 0 0 0 0 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 1 1 1
 0 0 0 1 0 0 0 0 0 0 0 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 1 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 1 1 0 0 1 0 0 0 0 1 0
 0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 1 0 0 1 0 0 0 0 1 1 1 0 0 1
 0 1 0 1 0 0 0 0 0 0 0 0 0 1 1 0 1 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 1 0 1
 0 0 0 1 0 1 0 0 0 0 0 0 0 0 1 0 0 1 0 0 1 0 0 0 0 0 1 0 0 0 0 1 1 0 1 1 0
 0 0 1 1 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 1 0 1 0
 0 0 1 1 0 0 0 0 0 0 0 0 1 0 0 1 0 1 0 0 0 1 0 0 0 1 0 0 0 0 0 0 1 0 0 0 1
 0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 1 0 0 1 1 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0
 0 1 1 0 0 0 0 0 0 1 0 0 1 0 0 0 1 1 1 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 1 0
 0 0 0 0 0 0 0 0 0 0 1 0 0 1 0 0 0 1 0 0 0 0 1 1 0 1 0 0 1 0 0 0 0 0 0 0 0
 1 0 0 0 0 1 0 0 0 0 0 0 

# Boosting

In [ ]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.linear_model import LogisticRegression

In [ ]:
boosting = AdaBoostClassifier(base_estimator=LogisticRegression(random_state = 0, 
                                                                C=10,penalty='l2', solver = 'liblinear', 
                                                                class_weight="balanced"),n_estimators=100)

boosting.fit(X_train, y_train)
y_pred_boost = boosting.predict(x_test)

In [ ]:
print(y_pred_boost)

[0 1 0 0 0 0 1 0 1 0 0 0 0 0 0 1 0 0 0 0 1 1 0 0 0 1 1 1 0 0 0 0 1 0 0 0 1
 0 1 1 0 1 1 0 0 1 0 0 1 0 0 1 0 0 1 0 0 0 1 0 0 0 1 0 1 0 1 0 0 0 0 1 1 0
 0 0 0 1 1 0 1 1 0 1 0 1 1 0 0 0 0 1 0 0 0 0 0 0 0 1 0 0 1 0 0 0 1 0 1 0 1
 0 0 1 1 0 0 1 0 0 0 0 0 1 1 0 0 1 0 1 0 0 1 0 0 0 0 0 0 1 0 0 0 1 0 0 0 0
 0 0 1 0 0 0 1 0 0 1 0 1 0 0 0 0 1 0 1 0 0 0 1 0 0 0 1 1 0 0 1 0 0 0 1 1 0
 1 0 0 0 0 0 0 1 0 0 0 1 0 0 0 0 1 0 0 0 0 1 0 1 1 0 1 0 0 0 0 1 1 1 0 1 1
 0 1 0 0 1 1 0 0 0 0 0 1 0 1 1 1 0 0 0 1 0 0 0 0 0 1 0 1 1 1 0 0 0 1 1 1 1
 0 0 0 1 1 1 0 0 1 0 0 1 0 1 0 1 1 1 1 0 1 0 0 0 0 0 1 0 0 0 0 1 0 0 1 1 0
 1 1 0 1 0 0 0 1 0 1 0 0 1 0 0 0 1 0 0 1 0 1 0 1 1 0 0 0 1 0 0 0 0 1 0 1 0
 0 0 1 1 0 0 0 0 0 0 0 0 1 0 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 1 0 0 0
 1 0 0 0 0 1 0 0 0 0 0 1 0 0 0 1 1 0 1 1 1 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0
 0 1 1 0 0 1 0 0 0 1 0 0 1 0 0 0 1 1 1 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 1 0
 0 1 0 0 0 0 0 0 1 0 1 0 0 0 0 1 0 1 0 0 0 0 0 1 0 1 0 0 1 0 0 0 0 0 0 0 0
 1 0 0 0 0 1 0 0 0 0 0 1 

# DETECCIÓN NIVEL DE TOXICIDAD

In [ ]:
y_nivel_train = df['toxicity_level']
X_nivel_train = tfidf

# Bagging

In [ ]:
from sklearn.ensemble import BaggingClassifier

m_bagging = BaggingClassifier(base_estimator=LogisticRegression(random_state = 0, C=10,penalty='l2'
                                 ,solver = 'liblinear',class_weight="balanced"),
                               n_estimators=10)

m_bagging.fit(X_nivel_train, y_nivel_train)
y_pred_bag_level = m_bagging.predict(x_test)

print(y_pred_bag_level)

[0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 2 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 0 0 0 2 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0
 0 0 0 2 0 0 0 0 0 0 0 1 1 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 2 0 1
 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 2 0 0 0 0 0 0 0 1 0 2 0 0 1
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1
 0 0 0 1 0 1 0 0 1 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0
 1 0 0 1 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 0 1 0
 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 1 0 1 0 0 0 1 0 0 0 0 0 0 0 0 1 0 1 0 0 0 1
 0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 1 0 0 2 2 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0
 0 2 2 0 0 0 0 0 0 1 0 0 0 0 0 0 1 1 1 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 1 0
 0 0 0 0 0 0 0 0 0 0 2 0 0 1 0 0 0 0 0 0 0 0 0 1 0 1 0 0 1 0 0 0 0 0 0 0 0
 1 0 0 0 0 1 0 0 0 0 0 0 

# Boosting

In [ ]:
from sklearn.ensemble import AdaBoostClassifier

boosting = AdaBoostClassifier(base_estimator=LogisticRegression(random_state = 0, C=10,penalty='l2'
,solver = 'liblinear',class_weight="balanced"),n_estimators=100)

boosting.fit(X_nivel_train, y_nivel_train)

y_pred_boost_level = boosting.predict(x_test)

print(y_pred_boost_level)

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1
 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 2 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 2 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 2 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 

In [ ]:
#test.head()
test.loc[:,'comment_id'].head()

0    10_001
1    10_002
2    10_003
3    10_004
4    10_005
Name: comment_id, dtype: object

# RESULTADOS

In [ ]:
import csv
df_logistic_toxicity = pd.DataFrame()
df_bagging_toxicity = pd.DataFrame()
df_boosting_toxicity = pd.DataFrame()

df_bagging_levels = pd.DataFrame()
df_boosting_levels = pd.DataFrame()


df_logistic_toxicity['toxicitylevel_number'] = y_pred_log
df_logistic_toxicity['id_comment'] = test.loc[:,'comment_id']

df_bagging_toxicity['toxicitylevel_number'] = y_pred_bag
df_bagging_toxicity['id_comment'] = test.loc[:,'comment_id']

df_boosting_toxicity['toxicitylevel_number'] = y_pred_boost
df_boosting_toxicity['id_comment'] = test.loc[:,'comment_id']



df_bagging_levels['toxicitylevel_number'] = y_pred_bag_level
df_bagging_levels['id_comment'] = test.loc[:,'comment_id']

df_boosting_levels['toxicitylevel_number'] = y_pred_boost_level
df_boosting_levels['id_comment'] = test.loc[:,'comment_id']




df_logistic_toxicity.to_csv('DetoxisLuciaYNora_subtask1_1.tsv', sep = '\t')
df_bagging_toxicity.to_csv('DetoxisLuciaYNora_subtask1_2.tsv', sep = '\t')
df_boosting_toxicity.to_csv('DetoxisLuciaYNora_subtask1_3.tsv', sep = '\t')

df_bagging_levels.to_csv('DetoxisLuciaYNora_subtask2_1.tsv', sep = '\t')
df_boosting_levels.to_csv('DetoxisLuciaYNora_subtask2_2.tsv', sep = '\t')